In [ ]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import f1_score, mean_squared_error
from scipy.stats import randint
import numpy as np

In [ ]:
# Load Wine dataset (3-class classification)
wine = load_wine()
X, y = wine.data, wine.target

In [ ]:
# Classification split (stratified)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
f1_dt = f1_score(y_test, y_pred_dt, average="weighted")

# Random Forest Classifier
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
f1_rf = f1_score(y_test, y_pred_rf, average="weighted")

print("Decision Tree F1 (weighted):", f1_dt)
print("Random Forest F1 (weighted):", f1_rf)


Decision Tree F1 (weighted): 0.9439974457215836
Random Forest F1 (weighted): 1.0


# **Classification explanation**
The Random Forest Classifier achieved a higher weighted F1 score than the single Decision Tree Classifier, so it performed better on the Wine classification task.

Random Forest tends to outperform a single tree because it combines predictions from many trees trained on different bootstrap samples and feature subsets, which reduces overfitting and variance while keeping bias relatively low.

In [ ]:
# Regression target (numeric version of class labels)
y_reg = y.astype(float)

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X, y_reg, test_size=0.2, random_state=42
)

# Decision Tree Regressor
dt_reg = DecisionTreeRegressor(random_state=42)
dt_reg.fit(X_train_r, y_train_r)
y_pred_dt_r = dt_reg.predict(X_test_r)
mse_dt = mean_squared_error(y_test_r, y_pred_dt_r)

# Random Forest Regressor [web:13]
rf_reg = RandomForestRegressor(random_state=42)
rf_reg.fit(X_train_r, y_train_r)
y_pred_rf_r = rf_reg.predict(X_test_r)
mse_rf = mean_squared_error(y_test_r, y_pred_rf_r)

print("DT Regressor MSE:", mse_dt)
print("RF Regressor MSE:", mse_rf)


DT Regressor MSE: 0.16666666666666666
RF Regressor MSE: 0.06483333333333333


In [ ]:
from scipy.stats import randint

param_dist_reg = {
    "n_estimators": randint(50, 300),          # number of trees
    "max_depth": [None, 5, 10, 20, 30],        # tree depth
    "min_samples_split": randint(2, 11)        # min samples split
}

rf_reg_base = RandomForestRegressor(random_state=42)

rand_search_reg = RandomizedSearchCV(
    estimator=rf_reg_base,
    param_distributions=param_dist_reg,
    n_iter=20,                     # number of random combos
    cv=5,
    scoring="neg_mean_squared_error",
    random_state=42,
    n_jobs=-1
)

rand_search_reg.fit(X_train_r, y_train_r)

print("Best RF regressor params:", rand_search_reg.best_params_)
best_rf_reg = rand_search_reg.best_estimator_

# Evaluate tuned RF regressor
y_pred_rf_best_r = best_rf_reg.predict(X_test_r)
mse_rf_best = mean_squared_error(y_test_r, y_pred_rf_best_r)
print("Tuned RF Regressor MSE:", mse_rf_best)


Best RF regressor params: {'max_depth': 20, 'min_samples_split': 3, 'n_estimators': 299}
Tuned RF Regressor MSE: 0.061069208770968254


# **Regression explanation**

The Random Forest Regressor achieved a lower mean squared error (MSE) than the Decision Tree Regressor, so it performed better at predicting the numeric labels.

After hyperparameter tuning with RandomizedSearchCV, the tuned Random Forest Regressor obtained an even smaller MSE than the default model, because choosing better values for n_estimators, max_depth, and min_samples_split improved the balance between underfitting and overfitting.